<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-sign-language-recognition-in-video/blob/main/NN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import math
import sys
import cv2
import io

from google.colab import drive

In [2]:
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/SU2/data"

Mounted at /content/drive


In [3]:
!git clone https://github.com/jiri99/SU_sign-language.git

Cloning into 'SU_sign-language'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 5), reused 8 (delta 2), pack-reused 0
Receiving objects: 100% (14/14), 6.44 KiB | 6.44 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [4]:
repo_path = "/content/SU_sign-language/"
sys.path.append(os.path.abspath(repo_path))

In [5]:
from buffer import ParquetData

In [6]:
selected_landmark_indices = [33, 133, 159, 263, 46, 70, 4, 454, 234, 10, 338, 297, 332, 61, 291, 0, 78, 14, 317,
                             152, 155, 337, 299, 333, 69, 104, 68, 398]

df_train = pd.read_csv(path + "/train_mod.csv", sep=",")
df_train.head()

data_load = ParquetData()
data_load.read_all(path, df_train, selected_landmark_indices)

Successfully read file from:  /content/drive/MyDrive/Colab Notebooks/SU2/data/train_landmark_files/2044/2946488174.parquet
Successfully read file from:  /content/drive/MyDrive/Colab Notebooks/SU2/data/train_landmark_files/2044/2923611079.parquet
Successfully read file from:  /content/drive/MyDrive/Colab Notebooks/SU2/data/train_landmark_files/2044/2930290203.parquet
Successfully read file from:  /content/drive/MyDrive/Colab Notebooks/SU2/data/train_landmark_files/2044/2930571096.parquet
Successfully read file from:  /content/drive/MyDrive/Colab Notebooks/SU2/data/train_landmark_files/2044/2929097553.parquet
Successfully read file from:  /content/drive/MyDrive/Colab Notebooks/SU2/data/train_landmark_files/2044/2923623190.parquet


KeyboardInterrupt: ignored

In [12]:
data_load.data["water"]["2044"].tensor.shape

(70, 70, 420)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed

# Assuming each frame is preprocessed into a 64x64 image with 1 channel (e.g., grayscale)
# And assuming we have 30 frames in each video
frame_height = 70
frame_width = 70
channels = 1
sequence_length = 420

# CNN model for spatial feature extraction
cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(frame_height, frame_width, channels)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten()
])

# Final model
model = Sequential([
    TimeDistributed(cnn, input_shape=(sequence_length, frame_height, frame_width, channels)),
    LSTM(64, return_sequences=False),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # Assuming 10 classes for the classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Now you can fit this model on your preprocessed data.
# X_train would be your video data with shape (num_samples, sequence_length, frame_height, frame_width, channels)
# Y_train would be your labels
# model.fit(X_train, Y_train, epochs=10, batch_size=32)



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 420, 6272)         92672     
 ributed)                                                        
                                                                 
 lstm (LSTM)                 (None, 64)                1622272   
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 1719754 (6.56 MB)
Trainable params: 1719754 (6.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
